1. Para desarrollar este desafío necesitarás los siguientes archivos:
 incidents.pkl
 officers.pkl
 subjects.pk

In [1]:
#para comentar importamos tanto pandas como los datos que estan en el mismo folder
import pandas as pd

# Cargar los datos
incidents = pd.read_pickle('incidents.pkl')
officers = pd.read_pickle('officers.pkl')
subjects = pd.read_pickle('subjects.pkl')

In [4]:
#revisamos las cabeceras para realizar el merge
print("Columnas de incidents:", incidents.columns)
print("Columnas de officers:", officers.columns)
print("Columnas de subjects:", subjects.columns)

Columnas de incidents: Index(['case_number', 'date', 'location', 'subject_statuses', 'subject_weapon',
       'subjects', 'subject_count', 'officers', 'officer_count',
       'grand_jury_disposition', 'attorney_general_forms_url', 'summary_url',
       'summary_text', 'latitude', 'longitude'],
      dtype='object')
Columnas de officers: Index(['case_number', 'race', 'gender', 'last_name', 'first_name',
       'full_name'],
      dtype='object')
Columnas de subjects: Index(['case_number', 'race', 'gender', 'last_name', 'first_name',
       'full_name'],
      dtype='object')


In [6]:
# Unir los DataFrames
merged_df = pd.merge(pd.merge(incidents, officers, on='case_number', suffixes=('_incident', '_officer')), 
                     subjects, on='case_number', suffixes=('_officer', '_subject'))


In [7]:
#ahora limpiaremos los datos verificando y eliminando los duplicados
merged_df = merged_df.drop_duplicates()

In [8]:
# Contamos sujetos de género F
num_female_subjects = merged_df['gender_subject'].value_counts()['F']
print("Número de sujetos de género F:", num_female_subjects)


Número de sujetos de género F: 9


In [10]:
#Obtener números de caso con al menos una sospechosa mujer
cases_with_female_suspects = merged_df[merged_df['gender_subject'] == 'F']['case_number'].unique()
num_cases_with_female_suspects = len(cases_with_female_suspects)
print("Número de casos con al menos una sospechosa mujer:", num_cases_with_female_suspects)

Número de casos con al menos una sospechosa mujer: 7


In [11]:
# Generar tabla pivote
pivot_table_gender = pd.pivot_table(merged_df, index='gender_officer', columns='gender_subject', aggfunc='size', fill_value=0)
print("Tabla pivote de género del oficial y del sujeto:")
print(pivot_table_gender)

Tabla pivote de género del oficial y del sujeto:
gender_subject  F    M
gender_officer        
F               2   18
M               7  350


# Para continuar con el desarrollo de este desafío, necesitarás el archivosCleaned_DS_Jobs.csv

In [12]:
# Cargamos los datos
jobs_df = pd.read_csv('Cleaned_DS_Jobs.csv')

# Reemplazamos los valores nulos
null_values = ["na", "NA", -1, "0", "-1", "null", "n/a", "N/A", "NULL"]
jobs_df.replace(null_values, pd.NA, inplace=True)

# Eliminamos las filas con datos faltantes
jobs_df.dropna(inplace=True)

In [13]:
# Generamos columnas de salario estimado mínimo y máximo
jobs_df[['Salario Estimado Mínimo', 'Salario Estimado Máximo']] = jobs_df['Salary Estimate'].str.split('-', expand=True)
jobs_df['Salario Estimado Mínimo'] = jobs_df['Salario Estimado Mínimo'].str.extract('(\d+)')
jobs_df['Salario Estimado Máximo'] = jobs_df['Salario Estimado Máximo'].str.extract('(\d+)')
jobs_df[['Salario Estimado Mínimo', 'Salario Estimado Máximo']] = jobs_df[['Salario Estimado Mínimo', 'Salario Estimado Máximo']].astype(float)

In [14]:
# Recodificmos la columna 'Size' usando mapping
size_mapping = {
    '10000+ employees': 'Mega Empresas',
    '5001 to 10000 employees': 'Grandes Empresas',
    '1001 to 5000 employees': 'Medianas Empresas',
    '201 to 500 employees': 'Pequeñas Empresas',
    '51 to 200 employees': 'Pequeñas Grandes Empresas',
    '501 to 1000 employees': 'Microempresas',
    'Unknown': 'Empresas sin Información'
}
jobs_df['Size'] = jobs_df['Size'].replace(size_mapping)

In [15]:
# para finalizar resumimos y reorganizamos con una tabla pivote
pivot_table_size = pd.pivot_table(jobs_df, index='Size', values=['Salario Estimado Mínimo', 'Salario Estimado Máximo'], aggfunc='mean')
print("Tabla pivote por tamaño de empresa:")
print(pivot_table_size)

Tabla pivote por tamaño de empresa:
                           Salario Estimado Máximo  Salario Estimado Mínimo
Size                                                                       
Empresas sin Información                110.500000                73.000000
Grandes Empresas                        138.875000                92.125000
Medianas Empresas                       137.461538                93.923077
Mega Empresas                           151.111111                97.888889
Microempresas                           146.235294               100.176471
Pequeñas Empresas                       141.142857                93.571429
Pequeñas Grandes Empresas               137.666667               100.666667
